In [1]:
def args_to_str(args:str, var:dict={'SAVED_EMBEDDINGS_DIR':"/vol/bitbucket/jq619/individual-project/saved_embeddings"}):
    for k, v in var.items():
        args = args.replace('$'+k, v)
    args = args.split(' ')[2:]
    args = [f"\"{arg}\"" for arg in args]
    args = ", \n".join(args)
    print(args)

In [2]:
args = 'python ./finetune.py --vision_model $VISION_MODEL --text_model $TEXT_MODEL --batch_size 128 --data_pct 1.0 --num_workers 1 --num_hidden_layers 1 --num_train_epochs 30 --seed 42 --lr 1e-4 --output_dir $SAVED_MODEL_DIR/${VISION_MODEL}_${TEXT_MODEL} --crop_size 256'
args_to_str(args)

"--vision_model", 
"$VISION_MODEL", 
"--text_model", 
"$TEXT_MODEL", 
"--batch_size", 
"128", 
"--data_pct", 
"1.0", 
"--num_workers", 
"1", 
"--num_hidden_layers", 
"1", 
"--num_train_epochs", 
"30", 
"--seed", 
"42", 
"--lr", 
"1e-4", 
"--output_dir", 
"$SAVED_MODEL_DIR/${VISION_MODEL}_${TEXT_MODEL}", 
"--crop_size", 
"256"


In [20]:
import torch
import os

path = '/vol/bitbucket/jq619/individual-project/saved_embeddings/image_embeds/ResNetAE'
split = 'train'
factor = 2

tensor_names = sorted([f for f in os.listdir(os.path.join(path, split))], key=lambda x: int(x.split('_')[1].split('.')[0]))

num_tensors = len(tensor_names)
new_num_tensors = num_tensors // factor
new_tensor_names = [f'{split}_{i}.pt' for i in range(new_num_tensors)]

for i in range(new_num_tensors):
    t_names = tensor_names[i*factor:i*factor+2]
    print(t_names)
    t1, t2 = [torch.load(os.path.join(path, split, tensor_name),
                         map_location='cpu') for tensor_name in t_names]
    if isinstance(t1, dict):
        t = dict()
        for k, v in t1.items():
            t[k] = torch.cat([t1[k], t2[k]], dim=0)
    else:
        t = torch.cat([t1, t2], dim=0)
    torch.save(t, os.path.join(path, split, new_tensor_names[i]))
print(f'You should delete {split}_{new_num_tensors}.pt onwards.')

['train_0.pt', 'train_1.pt']
['train_2.pt', 'train_3.pt']
['train_4.pt', 'train_5.pt']
['train_6.pt', 'train_7.pt']
['train_8.pt', 'train_9.pt']
['train_10.pt', 'train_11.pt']
['train_12.pt', 'train_13.pt']
['train_14.pt', 'train_15.pt']
['train_16.pt', 'train_17.pt']
['train_18.pt', 'train_19.pt']
['train_20.pt', 'train_21.pt']
['train_22.pt', 'train_23.pt']
['train_24.pt', 'train_25.pt']
['train_26.pt', 'train_27.pt']
['train_28.pt', 'train_29.pt']
['train_30.pt', 'train_31.pt']
['train_32.pt', 'train_33.pt']
['train_34.pt', 'train_35.pt']
['train_36.pt', 'train_37.pt']
['train_38.pt', 'train_39.pt']
['train_40.pt', 'train_41.pt']
['train_42.pt', 'train_43.pt']
['train_44.pt', 'train_45.pt']
['train_46.pt', 'train_47.pt']
['train_48.pt', 'train_49.pt']
['train_50.pt', 'train_51.pt']
['train_52.pt', 'train_53.pt']
['train_54.pt', 'train_55.pt']
['train_56.pt', 'train_57.pt']
['train_58.pt', 'train_59.pt']
['train_60.pt', 'train_61.pt']
['train_62.pt', 'train_63.pt']
['train_64.pt', 't

In [22]:
for j in range(new_num_tensors, num_tensors):
    os.remove(os.path.join(path, split, tensor_names[j]))

In [19]:
args = 'python3 ./pretrain.py --vision_model_type ae --vision_pretrained 101-elastic --text_pretrained dmis-lab/biobert-v1.1 --batch_size 128 --data_pct 0.01 --num_hidden_layers 1 --num_of_batches 100 --num_train_epochs 10 --seed 42 --text_embeds_raw_dir $SAVED_EMBEDDINGS_DIR/text_embeds/BioBERT --image_embeds_raw_dir $SAVED_EMBEDDINGS_DIR/image_embeds/ResNetAE --output_dir ./results/BioBERT_ResNetAE'
print(args_to_str(args))

"--vision_model_type", 
"ae", 
"--vision_pretrained", 
"101-elastic", 
"--text_pretrained", 
"dmis-lab/biobert-v1.1", 
"--batch_size", 
"128", 
"--data_pct", 
"0.01", 
"--num_hidden_layers", 
"1", 
"--num_of_batches", 
"100", 
"--num_train_epochs", 
"10", 
"--seed", 
"42", 
"--text_embeds_raw_dir", 
"/vol/bitbucket/jq619/individual-project/saved_embeddings/text_embeds/BioBERT", 
"--image_embeds_raw_dir", 
"/vol/bitbucket/jq619/individual-project/saved_embeddings/image_embeds/ResNetAE", 
"--output_dir", 
"./results/BioBERT_ResNetAE"


In [1]:
import torch 
import torch.nn as nn
from typing import List, Union, Tuple, Dict, Optional

import numpy as np

from transformers import AutoTokenizer
from transformers import BertModel, AutoModel, ViTImageProcessor

import torchxrayvision as xrv
from adaptor import Adaptor
from image_processor import ae_image_processor, timm_image_processor
from utils import load_timm_model

import skimage

d:\Apps\Anaconda3\envs\idv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_pretrained_available = [
    "bert-base-uncased", 
    "dmis-lab/biobert-v1.1", 
    "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", 
    "microsoft/BiomedVLP-CXR-BERT-general", 
]

# vision_model = xrv.autoencoders.ResNetAE(weights="101-elastic")

vision_model = load_timm_model('swin_base_patch4_window7_224', pretrained=True, retain_head=False)

# text_pretrained = "microsoft/BiomedVLP-CXR-BERT-general"
text_pretrained = "./weights/ClinicalBERT_checkpoint/ClinicalBERT_pretraining_pytorch_checkpoint"
text_model = BertModel.from_pretrained(text_pretrained)

model = Adaptor(
    text_model=text_model,
    vision_model=vision_model,
    vision_model_type='timm', 
    vision_output_dim=1024,
    projection_dim=768,
)

Some weights of the model checkpoint at ./weights/ClinicalBERT_checkpoint/ClinicalBERT_pretraining_pytorch_checkpoint were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
tokenizer = AutoTokenizer.from_pretrained(text_pretrained)
image_processor = lambda x: ViTImageProcessor()(x, return_tensors="pt", return_dict=True)

img_path = 'sample.jpeg'
img = skimage.io.imread(img_path)
imgs = np.stack([img, img])
vision_inputs = image_processor(imgs)

In [4]:
text_inputs = tokenizer(
    text=["Nodule", "Lung Lesion"], 
    return_tensors="pt", padding=True, 
)
inputs = {**vision_inputs, **text_inputs}
outputs = model(**inputs, return_dict=True, return_loss=True)

In [5]:
outputs

CLIPOutput(loss=tensor(0.6970, grad_fn=<DivBackward0>), logits_per_image=tensor([[5.6564, 5.7795],
        [5.4444, 5.5670]], grad_fn=<PermuteBackward0>), logits_per_text=tensor([[5.6564, 5.4444],
        [5.7795, 5.5670]], grad_fn=<MulBackward0>), text_embeds=tensor([[[-0.0491,  0.0511,  0.0189,  ...,  0.0209, -0.0117, -0.0364],
         [-0.0354,  0.0553,  0.0241,  ...,  0.0145, -0.0324, -0.0109],
         [-0.0534,  0.0547,  0.0104,  ...,  0.0183, -0.0111, -0.0345],
         [-0.0395,  0.0689,  0.0027,  ..., -0.0001, -0.0232, -0.0713],
         [-0.0561,  0.0393,  0.0314,  ...,  0.0209, -0.0105, -0.0294]],

        [[-0.0546,  0.0490,  0.0094,  ...,  0.0255, -0.0133, -0.0398],
         [-0.0497,  0.0555,  0.0149,  ...,  0.0104, -0.0076, -0.0357],
         [-0.0549,  0.0562,  0.0238,  ...,  0.0292, -0.0048, -0.0395],
         [-0.0454,  0.0566,  0.0048,  ...,  0.0393, -0.0093, -0.0353],
         [-0.0412,  0.0719, -0.0031,  ...,  0.0033, -0.0262, -0.0612]]],
       grad_fn=<DivBackwa